In [ ]:
import requests
import shutil
from bs4 import BeautifulSoup
import pandas as pd
import os

In [ ]:
def find_links(url, result):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    text_boxs = soup.find_all(attrs={"class": "textBox"})
    for box in text_boxs:
        result.append("https://irs.academy" + box.find("a")['href'])

In [ ]:
def find_images(url, image_links):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    image_boxes = soup.find_all(attrs={"class": "imgBox"})
    for image in image_boxes:
        image_links.append("https://irs.academy" + image.contents[5]["content"])

In [ ]:
urls = [
        "https://irs.academy/retush_and_photo",
        "https://irs.academy/business_category",
        "https://irs.academy/it_courses",
        "https://irs.academy/design_category",
        "https://irs.academy/needlework",
        "https://irs.academy/art_category",
        "https://irs.academy/exam_category",
        "https://irs.academy/beauty_category",
        "https://irs.academy/psychology_category",
        "https://irs.academy/inostrannye_yazyki_category",
        "https://irs.academy/drugoe_category"
]

In [ ]:
course_links = []
for url in urls:
    find_links(url, course_links)

In [ ]:
image_links = []
for url in urls:
    find_images(url, image_links)

for i, image in enumerate(image_links):
    r = requests.get(image, stream=True)
    if r.status_code == 200:
        with open("/content/drive/MyDrive/internship_bot/images/{}.jpg".format(i), 'wb') as f: 
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

In [ ]:
data = []

In [ ]:
for course in course_links:
    try:
        r = requests.get(course)
        soup = BeautifulSoup(r.content)
        text = ""
        for_whom = soup.find_all(attrs={"class": "for-whom-text"})
        for pers in for_whom:
            header = pers.find('h3').contents[0]
            discript = pers.find('p').contents[0]
            text += header + ":" + discript + os.linesep
        text += "После окончания курса вы сможете:"
        you = soup.find_all(attrs={"class": "you-can-item"})
        for item in you:
            text += item.find('p').contents[0] + ';'
        lessons = soup.find_all(attrs={"class": "lesson-descr"})
        for lesson in lessons:
            text += lesson.contents[0]
        
        header = soup.find('h1').contents[0]
        course = {"url":course, "header":header, "text":text}
        data.append(course)
    except:
        continue

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df

,url,header,text
0,https://irs.academy/videomontazh,Онлайн-курс видеомонтажа,Начинающих видеомонтажеров:Хотите освоить нову...
1,https://irs.academy/kurs_zvukooperator,Онлайн-курс звукооператора,Новичкам:Если вы хотите получить востребованну...
2,https://irs.academy/kurs_po_fashion_fotografii,Онлайн-курс по Fashion-фотографии,Производителей одежды:Которые хотят увеличения...
3,https://irs.academy/kurs_lightroom,Онлайн-курс по Lightroom,Для начинающих фотографов:Курс поможет вам сде...
4,https://irs.academy/kurs_po_youtube,Онлайн-курс по YouTube,Будущих блогеров:Хотите собрать большую аудито...
...,...,...,...
198,https://irs.academy/kurs_po_remontu,Онлайн-курс по ремонту,"Практичных хозяев:Вы догадываетесь, что иногда..."
199,https://irs.academy/kurs_po_remontu_bytivoi_te...,Онлайн-курс по ремонту бытовой техники,Начинающих мастеров:Вы уже занимаетесь починко...
200,https://irs.academy/kurs_po_remontu_stiralnyh_...,Онлайн-курс по ремонту стиральных машин,Мастеров по ремонту техники:Вы уже на практике...
201,https://irs.academy/kurs_po_remontu_telefonov,Онлайн-курс по ремонту телефонов,Начинающих мастеров:Уже занимаетесь починкой с...


In [ ]:
df.to_csv("text_data.csv")